In [58]:
import os
import pandas as pd
import numpy as np
import urllib.parse
import qrcode
import re
print ("Biblioteca importada com sucesso!")

Biblioteca importada com sucesso!


In [59]:
#Definir constantes para caminhos e URLs

URL_BASE_QR = "https://datacanuto.github.io/galeria-digital-artes"

# Obter diretório base do repositório (onde está o notebook)
DIRETORIO_BASE = os.path.dirname(os.path.abspath(__file__)) if '__file__' in globals() else os.getcwd()

# Definir caminhos relativos ao repositório
IMG_PATH = os.path.join(DIRETORIO_BASE, "assets", "img")
DIRETORIO_SAIDA = os.path.join(DIRETORIO_BASE, "links_obras")
QR_EXPORT_DIR = os.path.join(DIRETORIO_BASE, "qr_codes_export")
DADOS = os.path.join(DIRETORIO_BASE, "dados_obras.csv")
DADOS_TRATADOS = os.path.join(DIRETORIO_BASE, "obras_com_links.csv")

# Criar diretório para exportação de QR codes
os.makedirs(QR_EXPORT_DIR, exist_ok=True)
print(f"Diretório base: {DIRETORIO_BASE}")
print(f"Diretório de imagens: {IMG_PATH}")
print(f"Diretório de exportação de QR codes: {QR_EXPORT_DIR}")

Diretório base: c:\Users\pedro\Documents\qrCodeTelasExposicao\galeria-digital-artes
Diretório de imagens: c:\Users\pedro\Documents\qrCodeTelasExposicao\galeria-digital-artes\assets\img
Diretório de exportação de QR codes: c:\Users\pedro\Documents\qrCodeTelasExposicao\galeria-digital-artes\qr_codes_export


In [60]:
df = pd.read_csv(DADOS)
df = pd.DataFrame(df)
print("Dados carregados com sucesso!")
print("Total de registros:", len(df))
df.head()

Dados carregados com sucesso!
Total de registros: 62


,ITEM,TELAS,TÉCNICA,DIMENSÃO,ANO,VALOR (R$),VALOR_PIX,PARCELAMENTO,CATEGORIA
0,1,PORTAS E JANELAS 1,ACRÍLICA SOBRE TELA,100 X 60,2018,"6.000,00","5.700,00",5% À VISTA/ 6 VEZES,TELAS
1,2,PORTAS E JANELAS 2,ACRÍLICA SOBRE TELA,70 X 120,2024,"8.400,00","7.980,00",5% À VISTA/ 6 VEZES,TELAS
2,3,PORTAS E JANELAS 3,ACRÍLICA SOBRE TELA,53 X 73,2024,"3.860,00","3.667,00",5% À VISTA/ 6 VEZES,TELAS
3,4,PORTAS E JANELAS 4,ACRÍLICA SOBRE TELA,50 X 70,2024,"3.500,00","3.325,00",5% À VISTA/ 6 VEZES,TELAS
4,5,PORTAS E JANELAS 5,ACRÍLICA SOBRE TELA,70 X 100,2024,"7.000,00","6.650,00",5% À VISTA/ 6 VEZES,TELAS


In [61]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62 entries, 0 to 61
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ITEM          62 non-null     int64 
 1   TELAS         62 non-null     object
 2   TÉCNICA       62 non-null     object
 3   DIMENSÃO      62 non-null     object
 4   ANO           62 non-null     int64 
 5   VALOR (R$)    62 non-null     object
 6   VALOR_PIX     62 non-null     object
 7   PARCELAMENTO  62 non-null     object
 8   CATEGORIA     62 non-null     object
dtypes: int64(2), object(7)
memory usage: 4.5+ KB


In [62]:
df.columns = df.columns.str.strip()
df.columns = df.columns.str.lower()

print(df.columns)

Index(['item', 'telas', 'técnica', 'dimensão', 'ano', 'valor (r$)',
       'valor_pix', 'parcelamento', 'categoria'],
      dtype='object')


In [63]:
img_list = os.listdir(IMG_PATH)
img_fullpath = [os.path.join(IMG_PATH, img) for img in img_list]

# Criar dicionário para mapear índice com imagem
img_dict = {i+1: img for i, img in enumerate(img_list)}
df['fotos'] = df['item'].map(img_dict)

In [64]:
# Limpar e converter colunas de valor para float
# Substituir 'acervo pessoal', '-' e valores vazios por 0

def limpar_valor_brasileiro(valor):
    """Converte valores no formato brasileiro (ex: '6.000,00') para float"""
    if pd.isna(valor):
        return 0.0
    
    valor_str = str(valor).strip().lower()
    
    # Verificar se é 'acervo pessoal', vazio ou '-'
    if valor_str in ['acervo pessoal', '', 'nan', '-', '  -   ']:
        return 0.0
    
    # Remover espaços
    valor_str = valor_str.replace(' ', '')
    
    # Formato brasileiro: "6.000,00" 
    # Remover pontos de milhar e trocar vírgula por ponto decimal
    if ',' in valor_str:
        # Se tem vírgula, é o separador decimal
        valor_str = valor_str.replace('.', '')  # Remove pontos (milhares)
        valor_str = valor_str.replace(',', '.')  # Vírgula vira ponto decimal
    # Se não tem vírgula mas tem ponto, assumir que já está correto
    
    try:
        valor_float = float(valor_str)
        # Verificar se o valor está muito alto (multiplicado por 100)
        # Valores normais de arte: 350 a 10.000 reais
        if valor_float > 100000:  # Se maior que 100k, dividir por 100
            valor_float = valor_float / 100
        return round(valor_float, 2)
    except:
        return 0.0

# Processar coluna 'valor (r$)'
df['valor (r$)'] = df['valor (r$)'].apply(limpar_valor_brasileiro)

# Processar coluna 'valor_pix' (se existir)
if 'valor_pix' in df.columns:
    df['valor_pix'] = df['valor_pix'].apply(limpar_valor_brasileiro)

print("Colunas de valor convertidas para float com 2 casas decimais!")
print("\nExemplo dos dados:")

if 'valor_pix' in df.columns:
    print(df[['telas', 'valor (r$)', 'valor_pix']].head(15))
else:
    print(df[['telas', 'valor (r$)']].head(15))

Colunas de valor convertidas para float com 2 casas decimais!

Exemplo dos dados:
                  telas  valor (r$)  valor_pix
0    PORTAS E JANELAS 1      6000.0     5700.0
1    PORTAS E JANELAS 2      8400.0     7980.0
2    PORTAS E JANELAS 3      3860.0     3667.0
3    PORTAS E JANELAS 4      3500.0     3325.0
4   PORTAS E JANELAS 5       7000.0     6650.0
5       CASA CURITIBA 1      6720.0     6384.0
6       CASA CURITIBA 2      6720.0     6384.0
7        FUSCA VERMELHO      3500.0     3325.0
8               WALKING      7500.0     7125.0
9                SUCATA         0.0        0.0
10         STREET DANCE         0.0        0.0
11              PILATES      3500.0     3325.0
12            OLD TIMES      3500.0     3325.0
13        CENAS URBANAS      4800.0     4560.0
14          FERRO VELHO      6720.0     6384.0


In [65]:
# Criar coluna 'status' [disponível, acervo pessoal]
df['status'] = np.where(df['valor (r$)'] != 0, 'disponível', 'acervo pessoal')

In [66]:
# Criar coluna 'op_parcelamento' extraindo número inteiro após a barra '/'
df['op_parcelamento'] = df['parcelamento'].apply(lambda x: re.search(r'/\s*(\d+)', str(x)).group(1) if pd.notna(x) and re.search(r'/\s*(\d+)', str(x)) else '0')

In [67]:
# Carregar links do CSV gerado pela API (Crédito)
csv_com_links = os.path.join(DIRETORIO_BASE, "obras_com_links_api.csv")

if os.path.exists(csv_com_links):
    df_links = pd.read_csv(csv_com_links)
    df_links.columns = df_links.columns.str.strip().str.lower()
    df['link_mp'] = df_links['link_mp']
    print(f"✅ {len([l for l in df['link_mp'] if pd.notna(l) and l != 'None'])} links de crédito carregados")
    print(f"✅ {len([l for l in df['link_mp'] if pd.isna(l) or l == 'None'])} obras no acervo pessoal")
    
else:
    print(f"⚠️  Arquivo {csv_com_links} não encontrado!")
    print("Execute primeiro: .\\venv\\Scripts\\python.exe mercado_pago_api.py --mode prod")



✅ 60 links de crédito carregados
✅ 2 obras no acervo pessoal


In [68]:
# Carregar links PIX do CSV gerado pela API
csv_com_links_pix = os.path.join(DIRETORIO_BASE, "obras_com_links_pix_api.csv")

if os.path.exists(csv_com_links_pix):
    df_links_pix = pd.read_csv(csv_com_links_pix)
    df_links_pix.columns = df_links_pix.columns.str.strip().str.lower()
    df['link_mp_pix'] = df_links_pix['link_mp_pix']
    print(f"✅ {len([l for l in df['link_mp_pix'] if pd.notna(l) and l != 'None'])} links PIX carregados")
else:
    print(f"⚠️  Arquivo {csv_com_links_pix} não encontrado!")
    print("Execute primeiro: .\\venv\\Scripts\\python.exe mercado_pago_api_pix.py --mode prod")

print(f"\nExemplo link crédito: {df['link_mp'].iloc[0]}")
print(f"Exemplo link PIX: {df['link_mp_pix'].iloc[0]}")

✅ 60 links PIX carregados

Exemplo link crédito: https://www.mercadopago.com.br/checkout/v1/redirect?pref_id=3041576384-582408e8-e3de-487f-bf63-59d884161692
Exemplo link PIX: https://www.mercadopago.com.br/checkout/v1/redirect?pref_id=3041576384-c8c00105-8fe7-444b-9466-97d6dce8f6dc


In [69]:
df.head()

,item,telas,técnica,dimensão,ano,valor (r$),valor_pix,parcelamento,categoria,fotos,status,op_parcelamento,link_mp,link_mp_pix
0,1,PORTAS E JANELAS 1,ACRÍLICA SOBRE TELA,100 X 60,2018,6000.0,5700.0,5% À VISTA/ 6 VEZES,TELAS,REF 01...PORTAS E JANELAS 1.jpg,disponível,6,https://www.mercadopago.com.br/checkout/v1/red...,https://www.mercadopago.com.br/checkout/v1/red...
1,2,PORTAS E JANELAS 2,ACRÍLICA SOBRE TELA,70 X 120,2024,8400.0,7980.0,5% À VISTA/ 6 VEZES,TELAS,REF 02...PORTAS E JANELAS 2.jpg,disponível,6,https://www.mercadopago.com.br/checkout/v1/red...,https://www.mercadopago.com.br/checkout/v1/red...
2,3,PORTAS E JANELAS 3,ACRÍLICA SOBRE TELA,53 X 73,2024,3860.0,3667.0,5% À VISTA/ 6 VEZES,TELAS,REF 03...PORTAS E JANELAS 3.jpg,disponível,6,https://www.mercadopago.com.br/checkout/v1/red...,https://www.mercadopago.com.br/checkout/v1/red...
3,4,PORTAS E JANELAS 4,ACRÍLICA SOBRE TELA,50 X 70,2024,3500.0,3325.0,5% À VISTA/ 6 VEZES,TELAS,REF 04 ...PORTAS E JANELAS 4.jpg,disponível,6,https://www.mercadopago.com.br/checkout/v1/red...,https://www.mercadopago.com.br/checkout/v1/red...
4,5,PORTAS E JANELAS 5,ACRÍLICA SOBRE TELA,70 X 100,2024,7000.0,6650.0,5% À VISTA/ 6 VEZES,TELAS,REF 05 - PORTAS E JANELAS 5.jpg,disponível,6,https://www.mercadopago.com.br/checkout/v1/red...,https://www.mercadopago.com.br/checkout/v1/red...


In [70]:
# Carregar template HTML único
TEMPLATE_OBRA = os.path.join(DIRETORIO_BASE, "assets", "template_obra.html")

with open(TEMPLATE_OBRA, 'r', encoding='utf-8') as f:
    template_obra = f.read()

In [71]:
# Funções auxiliares


def formatar_valor_brasileiro(valor):
    """Formata valor float para padrão brasileiro (ex: 1.234,56)"""
    return f"{valor:,.2f}".replace(',', 'X').replace('.', ',').replace('X', '.')


def gerar_html_obra(row):
    """Gera o HTML completo da obra substituindo placeholders"""
    titulo = row.get('telas', '')
    titulo_encoded = urllib.parse.quote(str(titulo))
    tecnica_dimensao_ano = f"{row.get('técnica', '')}, {row.get('dimensão', '')}, {row.get('ano','')}"
    
    valor_credito = row.get('valor (r$)', 0)
    valor_pix = row.get('valor_pix', 0)
    
    html = template_obra.replace('{{TITULO}}', titulo) \
                        .replace('{{FOTO}}', row.get('fotos', '')) \
                        .replace('{{TECNICA_DIMENSAO_ANO}}', tecnica_dimensao_ano) \
                        .replace('{{TITULO_ENCODED}}', titulo_encoded) \
                        .replace('{{URL_BASE}}', URL_BASE_QR)
    
    # Decidir se mostra pagamento ou acervo
    if valor_credito > 0 and valor_pix > 0:
        # Preencher dados de pagamento
        html = html.replace('{{LINK_CREDITO}}', row.get('link_mp', '#')) \
                   .replace('{{VALOR_CREDITO}}', formatar_valor_brasileiro(valor_credito)) \
                   .replace('{{OP_PARCELAMENTO}}', str(row.get('op_parcelamento', '6'))) \
                   .replace('{{LINK_PIX}}', row.get('link_mp_pix', '#')) \
                   .replace('{{VALOR_PIX}}', formatar_valor_brasileiro(valor_pix))
        
        # Remover bloco de acervo completo (incluindo marcadores)
        html = re.sub(r'\{\{#IF_ACERVO\}\}.*?\{\{#ENDIF\}\}', '', html, flags=re.DOTALL)
        
        # Remover marcadores de disponível
        html = html.replace('{{#IF_DISPONIVEL}}', '').replace('{{#ENDIF}}', '')
    else:
        # Remover bloco de pagamento completo (incluindo marcadores)
        html = re.sub(r'\{\{#IF_DISPONIVEL\}\}.*?\{\{#ENDIF\}\}', '', html, flags=re.DOTALL)
        
        # Remover marcadores de acervo
        html = html.replace('{{#IF_ACERVO}}', '').replace('{{#ENDIF}}', '')
    
    return html


def gerar_qrcode(link, caminho_salvar):
    """Gera e salva um QR code"""
    qr = qrcode.QRCode(
        version=1,
        error_correction=qrcode.constants.ERROR_CORRECT_L,
        box_size=10,
        border=4,
    )
    qr.add_data(link)
    qr.make(fit=True)
    imagem_qr = qr.make_image(fill_color="black", back_color="white")
    imagem_qr.save(caminho_salvar)

In [72]:
# Loop principal: Gerador de etiquetas
for i, row in df.iterrows():
    idx_val = int(row.get('item', i))
    pasta = f"item_{idx_val}"
    caminho_pasta = os.path.join(DIRETORIO_SAIDA, pasta)
    os.makedirs(caminho_pasta, exist_ok=True)
    
    link_final = f"{URL_BASE_QR}/links_obras/{pasta}/index.html"
    df.at[i, 'qr link'] = link_final
    
    # Gerar e salvar HTML
    html_content = gerar_html_obra(row)
    with open(os.path.join(caminho_pasta, "index.html"), "w", encoding="utf-8") as f:
        f.write(html_content)
    
    # Gerar QR codes
    gerar_qrcode(link_final, os.path.join(caminho_pasta, "qrcode.png"))
    
    qr_export_nome = f"qr_{idx_val}.png"
    gerar_qrcode(link_final, os.path.join(QR_EXPORT_DIR, qr_export_nome))
    df.at[i, 'qr_export_file'] = qr_export_nome

print("✅ Páginas HTML e QR Codes gerados com sucesso!")
print(f"📁 QR Codes exportados para: {QR_EXPORT_DIR}")
print(f"\n📊 Resumo:")
print(f"  - Obras disponíveis: {len(df[df['valor (r$)'] > 0])}")
print(f"  - Acervo pessoal: {len(df[df['valor (r$)'] == 0])}")

✅ Páginas HTML e QR Codes gerados com sucesso!
📁 QR Codes exportados para: c:\Users\pedro\Documents\qrCodeTelasExposicao\galeria-digital-artes\qr_codes_export

📊 Resumo:
  - Obras disponíveis: 60
  - Acervo pessoal: 2


In [73]:
# Salvar CSV com links de pagamento no repositório
csv_output = os.path.join(DIRETORIO_BASE, "obras_com_links.csv")
df.to_csv(csv_output, index=False, encoding="utf-8")
print(f"CSV salvo em: {csv_output}")
df.head()

CSV salvo em: c:\Users\pedro\Documents\qrCodeTelasExposicao\galeria-digital-artes\obras_com_links.csv


,item,telas,técnica,dimensão,ano,valor (r$),valor_pix,parcelamento,categoria,fotos,status,op_parcelamento,link_mp,link_mp_pix,qr link,qr_export_file
0,1,PORTAS E JANELAS 1,ACRÍLICA SOBRE TELA,100 X 60,2018,6000.0,5700.0,5% À VISTA/ 6 VEZES,TELAS,REF 01...PORTAS E JANELAS 1.jpg,disponível,6,https://www.mercadopago.com.br/checkout/v1/red...,https://www.mercadopago.com.br/checkout/v1/red...,https://datacanuto.github.io/galeria-digital-a...,qr_1.png
1,2,PORTAS E JANELAS 2,ACRÍLICA SOBRE TELA,70 X 120,2024,8400.0,7980.0,5% À VISTA/ 6 VEZES,TELAS,REF 02...PORTAS E JANELAS 2.jpg,disponível,6,https://www.mercadopago.com.br/checkout/v1/red...,https://www.mercadopago.com.br/checkout/v1/red...,https://datacanuto.github.io/galeria-digital-a...,qr_2.png
2,3,PORTAS E JANELAS 3,ACRÍLICA SOBRE TELA,53 X 73,2024,3860.0,3667.0,5% À VISTA/ 6 VEZES,TELAS,REF 03...PORTAS E JANELAS 3.jpg,disponível,6,https://www.mercadopago.com.br/checkout/v1/red...,https://www.mercadopago.com.br/checkout/v1/red...,https://datacanuto.github.io/galeria-digital-a...,qr_3.png
3,4,PORTAS E JANELAS 4,ACRÍLICA SOBRE TELA,50 X 70,2024,3500.0,3325.0,5% À VISTA/ 6 VEZES,TELAS,REF 04 ...PORTAS E JANELAS 4.jpg,disponível,6,https://www.mercadopago.com.br/checkout/v1/red...,https://www.mercadopago.com.br/checkout/v1/red...,https://datacanuto.github.io/galeria-digital-a...,qr_4.png
4,5,PORTAS E JANELAS 5,ACRÍLICA SOBRE TELA,70 X 100,2024,7000.0,6650.0,5% À VISTA/ 6 VEZES,TELAS,REF 05 - PORTAS E JANELAS 5.jpg,disponível,6,https://www.mercadopago.com.br/checkout/v1/red...,https://www.mercadopago.com.br/checkout/v1/red...,https://datacanuto.github.io/galeria-digital-a...,qr_5.png


In [74]:
# Gerar QR Code para o catálogo completo
catalogo_link = "https://datacanuto.github.io/galeria-digital-artes/catalog_mobile/index.html"

qr_catalogo = qrcode.QRCode(
    version=1,
    error_correction=qrcode.constants.ERROR_CORRECT_L,
    box_size=10,
    border=4,
)
qr_catalogo.add_data(catalogo_link)
qr_catalogo.make(fit=True)

imagem_qr_catalogo = qr_catalogo.make_image(fill_color="black", back_color="white")

# Salvar QR code do catálogo
qr_catalogo_path = os.path.join(QR_EXPORT_DIR, "qr_catalogo_completo.png")
imagem_qr_catalogo.save(qr_catalogo_path)

print(f"QR Code do catálogo gerado com sucesso!")
print(f"Salvo em: {qr_catalogo_path}")

QR Code do catálogo gerado com sucesso!
Salvo em: c:\Users\pedro\Documents\qrCodeTelasExposicao\galeria-digital-artes\qr_codes_export\qr_catalogo_completo.png
